In [1]:
import pandas as pd
import numpy as np
import copy

# initialize variables i will need

i = 0
totalProfit = 0.0
rawGroupDfs = {}
brokenDf = pd.DataFrame(columns=['Name', 'Total Bought', 'Total Sold', 'Profit', 'Fee'])
resultDf = pd.DataFrame(columns=['Name', 'Total Bought', 'Total Sold', 'Profit', 'Fee'])
df = pd.read_excel('TradeHistory.xlsx')

df.head(10)

,Date,Market,Type,Price,Amount,Total,Fee,Fee Coin
0,2018-02-10 19:54:28,ZECBTC,BUY,0.055998,0.052,0.002912,0.000402,BNB
1,2018-02-10 19:54:28,ZECBTC,BUY,0.055999,0.582,0.032591,0.004505,BNB
2,2018-02-10 19:53:06,OMGBTC,BUY,0.001412,0.550,0.000777,0.000107,BNB
3,2018-02-10 19:25:07,ETCBTC,BUY,0.002789,12.740,0.035532,0.004914,BNB
4,2018-02-10 17:53:10,XMRBTC,SELL,0.029272,1.224,0.035829,0.004926,BNB
5,2018-02-10 17:10:27,XMRBTC,BUY,0.029021,1.224,0.035522,0.004906,BNB
6,2018-02-10 15:43:04,SUBBTC,BUY,0.000086,412.000,0.035510,0.004775,BNB
7,2018-02-10 15:41:05,LTCBTC,BUY,0.018400,1.930,0.035512,0.004776,BNB
8,2018-02-10 15:36:28,POWRBTC,BUY,0.000089,398.000,0.035498,0.004766,BNB
9,2018-02-10 15:25:30,CNDBTC,BUY,0.000022,1495.000,0.032217,0.004356,BNB


In [2]:
xls = df.groupby(df['Market'])

#loop start - long cell bc all work being calculated in one go AKA O(n)
for name, group in xls:
    
    group = group.sort_values(by=['Date'], ascending=True)
    rawGroupDfs[name] = copy.deepcopy(group)
    
    # Create group dataframes for profit calculations
    group_df = pd.DataFrame(columns=['Total Bought', 'Total Sold', 'Profit', 'Fee'])
    

    ## Remove sells from top
    group = group.reset_index()
    for index, row in group.iterrows():
        if row['Type'] == 'SELL':
            group.drop(index, inplace=True)
        else:
            break

    ## Remove buys from bottom
    while group['Type'].count() > 0 and group.iloc[-1]['Type'] == 'BUY':
        group = group[:-1]


    #check for groups that only sold or bought, dont count them
    if len(group['Type'].unique()) == 1:
        continue
        

    
    ## Calculate profits, if the total amount of bought and sold coins match up -- (per group)
    
    totalCostForCurrentGroup = 0.0
    totalRevenueForCurrentGroup = 0.0
    totalFeeForCurrentGroup = 0.0
    totalAmountForBuys = 0.0
    totalAmountForSells = 0.0
    totalProfitForCurrentGroup = 0.0
    for index, row in group.iterrows():
        if row['Type'] == 'BUY':
            totalAmountForBuys = totalAmountForBuys + float(row['Amount'])
            totalCostForCurrentGroup = totalCostForCurrentGroup + float(row['Total'])
        if row['Type'] == 'SELL':
            totalAmountForSells = totalAmountForSells + float(row['Amount'])
            totalRevenueForCurrentGroup = totalRevenueForCurrentGroup + float(row['Total'])
        totalFeeForCurrentGroup = totalFeeForCurrentGroup + float(row['Fee'])
        totalProfitForCurrentGroup = totalRevenueForCurrentGroup - totalCostForCurrentGroup

    # sometimes there is a slight difference in the amount bought / sold 
    # (maybe limit orders didnt get fully filled in time)
    
    # We will recalculate profits to be conservative to the lower amount
    newTotalAmountForSells = 0.0
    newTotalAmountForBuys = 0.0
    totalProfitForCurrentGroup = totalRevenueForCurrentGroup - totalCostForCurrentGroup
    group_df = pd.DataFrame()
    if totalAmountForSells > totalAmountForBuys:
        group_df = group_df.set_value(i, 'Only counting', totalAmountForSells if totalAmountForBuys > totalAmountForSells else totalAmountForBuys)
        totalProfitForCurrentGroup = totalProfitForCurrentGroup * (totalAmountForBuys / totalAmountForSells)
        totalAmountForSells = totalAmountForSells * (totalAmountForBuys / totalAmountForSells)
        group['Only counting'] = totalAmountForBuys
        print('reduced profit for group ' + name)
        
    if totalAmountForSells < totalAmountForBuys:
        group_df = group_df.set_value(i, 'Only counting', totalAmountForSells if totalAmountForBuys > totalAmountForSells else totalAmountForBuys)        
        totalProfitForCurrentGroup = totalProfitForCurrentGroup * (totalAmountForSells / totalAmountForBuys)
        totalAmountForBuys = totalAmountForBuys * (totalAmountForSells / totalAmountForBuys)
        group['Only counting'] = totalAmountForSells
        print('reduced profit for group ' + name)




    group_df = group_df.set_value(i, 'Total Bought', totalAmountForBuys)
    group_df = group_df.set_value(i, 'Total Sold', totalAmountForSells)
    group_df = group_df.set_value(i, 'Profit', totalProfitForCurrentGroup)
    group_df = group_df.set_value(i, 'Fee', totalFeeForCurrentGroup)
    group_df = group_df.set_value(i, 'Name', name)
    if totalProfitForCurrentGroup > 0 :
        resultDf = resultDf.append(group_df)
    else: 
        brokenDf = brokenDf.append(group_df)
        if totalProfitForCurrentGroup == 0.0:
            print('Group ' + name + ' had no round trip trades in this time period')
        else:
            print('Problems calculating group ' + name + 
                  '. Total profit not added to final result is: ' + str(totalProfitForCurrentGroup))

    i = i + 1
    
#loop over

/Users/dimitrideychakiwsky/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:59: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/dimitrideychakiwsky/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:75: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/dimitrideychakiwsky/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/dimitrideychakiwsky/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:77: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/dimitrideychakiwsky/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:78: FutureWarning: set_value is deprecated and wi

reduced profit for group ADABTC
Group BTGBTC had no round trip trades in this time period
Group BTSBTC had no round trip trades in this time period
reduced profit for group ETHBTC
Group LUNBTC had no round trip trades in this time period
reduced profit for group SNTBTC
Group STRATBTC had no round trip trades in this time period
reduced profit for group XVGBTC
reduced profit for group ZECBTC


In [3]:
print('Successful calculation of below groups:')
resultDf.fillna('-')

Successful calculation of below groups:


,Fee,Name,Only counting,Profit,Total Bought,Total Sold
0,0.196967,ADABTC,4131,0.023622,4131.000,4131.000
1,0.402617,BCDBTC,-,0.007096,78.398,78.398
4,0.424939,CNDBTC,-,0.006785,20565.000,20565.000
5,0.027002,ETCBTC,-,0.001261,21.540,21.540
6,0.034367,ETHBTC,0.303,0.000518,0.303,0.303
7,0.031223,HSRBTC,-,0.001147,51.770,51.770
8,0.065395,IOTABTC,-,0.001510,358.000,358.000
9,0.090358,LTCBTC,-,0.001091,6.190,6.190
11,0.035530,MCOBTC,-,0.000594,33.250,33.250
12,0.194931,OMGBTC,-,0.002490,177.590,177.590


In [4]:
profitDf = pd.DataFrame(columns=['Total Profit BTC', 'Total Fees BNB'])
profitDf['Total Profit BTC'] = [resultDf['Profit'].sum()]
profitDf['Total Fees BNB'] = [resultDf['Fee'].sum()]
profitDf

,Total Profit BTC,Total Fees BNB
0,0.108422,3.185559


In [5]:
print('These were not added to final result. Bot never sells at a loss, need to find out why this is happening')
brokenDf

These were not added to final result. Bot never sells at a loss, need to find out why this is happening


,Fee,Name,Profit,Total Bought,Total Sold
2,0.0,BTGBTC,0.0,0.0,0.0
3,0.0,BTSBTC,0.0,0.0,0.0
10,0.0,LUNBTC,0.0,0.0,0.0
16,0.0,STRATBTC,0.0,0.0,0.0


In [6]:
#debugging purposes
#rawGroupDfs['SUBBTC']

In [7]:
#xls.get_group('SUBBTC')